In [1]:
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession

In [2]:
#read csv

testdf = pd.read_csv('datasets/test.csv')
traindf = pd.read_csv('datasets/train.csv')
alldf = pd.read_csv('datasets/submit.csv')

ytestdf = testdf.drop(['title', 'author', 'text'], axis=1)

# ytestdf['labels'] = np.zeros((len(testdf), 1))

In [3]:
for i, j in alldf.iterrows():
    for k, l in testdf.iterrows():
        if j['id'] == l['id']:
            ytestdf.loc[j['id'], 'label'] = j['label']

# ytestdf = ytestdf.merge(alldf, left_on='id', right_on='id', how='left')[['id', 'label']]
ytestdf = pd.merge(left = ytestdf, right=alldf, left_on='id', right_on='id', how='left')

In [4]:
# spark = SparkSession.builder.appName('fakenews').getOrCreate()
# print(spark)

In [5]:
# spark.stop()